# Random Perturbations

Streamlined notebook for evaluating random perturbations on saved models and datasets.

## Imports

In [1]:
# Standard library
import os
import sys
import time

# Third-party
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Local package imports
from minima_volume.perturb_funcs import ( analyze_wiggles_metrics )

from minima_volume.dataset_funcs import (
    load_dataset,
    load_model,
    load_models_and_data,
    prepare_datasets,
    tensor_to_list,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Input Parameters

In [2]:
# Perturbation Configuration
perturbation_seed = 1
num_directions = 500
N = 100
x = np.linspace(0, 1, N)
coefficients = x**2

# Other Configuration
dataset_quantities = [0, 600-60, 2000 - 60, 6000 - 60, 20000 - 60, 60000 - 60]
base_output_dir = ""

## Model + Dataset Specific Code

This is for model and dataset specific code.

In [3]:
# User specifies the model module name
from minima_volume.models import MNIST_model_data as model_module

# Generate dataset
#x_base, y_base, x_test, y_test = model_module.get_dataset(
#    device = device
#)

# MNIST specific initialization parameters
hidden_dims = [256, 128]

# Grab model
model_template = model_module.get_model(hidden_dims=hidden_dims, device=device, seed=0)

# Grab loss and metrics
loss_fn = model_module.get_loss_fn()
other_metrics = model_module.get_additional_metrics()

## Loading Model and Datasets

In [4]:
# ====================================
# Load Trained Models and Dataset
# ====================================
target_dir = "models_and_data"  # relative path
loaded_models, loaded_model_data, loaded_dataset = load_models_and_data(
    model_template=model_template,
    target_dir=target_dir,
    device=device,
)

# Dataset Info
dataset_type = loaded_dataset['dataset_type']
print(f"Dataset type: {dataset_type}")
print(f"Dataset quantities: {loaded_dataset['dataset_quantities']}")

print("\nTensor shapes:")
for key in ["x_base_train", "y_base_train", "x_additional", "y_additional", "x_test", "y_test"]:
    shape = getattr(loaded_dataset[key], "shape", None)
    print(f"  {key}: {shape if shape is not None else 'None'}")

# Reconstruct trained_model dicts safely.
# If the loss or accuracy or additional metrics happen to be
# tensors, they get safely converted to lists.
all_models = [
    {
        "model": model,
        **{
            k: tensor_to_list(model_data[k], key_path=k)
            for k in ["train_loss", "train_accs", "test_loss", "test_accs", "additional_data", "dataset_type"]
        },
    }
    for model, model_data in zip(loaded_models, loaded_model_data)
]
print(f"Reconstructed {len(all_models)} trained models")

Looking for models and dataset in: models_and_data
Found 6 model files:
  - model_additional_0.pt
  - model_additional_1940.pt
  - model_additional_19940.pt
  - model_additional_540.pt
  - model_additional_5940.pt
  - model_additional_59940.pt
✅ Model loaded into provided instance from models_and_data\model_additional_0.pt
Successfully loaded: model_additional_0.pt
✅ Model loaded into provided instance from models_and_data\model_additional_1940.pt
Successfully loaded: model_additional_1940.pt
✅ Model loaded into provided instance from models_and_data\model_additional_19940.pt
Successfully loaded: model_additional_19940.pt
✅ Model loaded into provided instance from models_and_data\model_additional_540.pt
Successfully loaded: model_additional_540.pt
✅ Model loaded into provided instance from models_and_data\model_additional_5940.pt
Successfully loaded: model_additional_5940.pt
✅ Model loaded into provided instance from models_and_data\model_additional_59940.pt
Successfully loaded: model_

✅ Dataset loaded from models_and_data\dataset.pt
Dataset type: data
Dataset quantities: [0, 540, 1940, 5940, 19940, 59940]

Tensor shapes:
  x_base_train: torch.Size([60, 1, 28, 28])
  y_base_train: torch.Size([60])
  x_additional: torch.Size([59940, 1, 28, 28])
  y_additional: torch.Size([59940])
  x_test: torch.Size([10000, 1, 28, 28])
  y_test: torch.Size([10000])
Reconstructed 6 trained models


## Perturbations

Using the saved datasets, we perform model perturbations. 

In [5]:
 

x_base_train = loaded_dataset['x_base_train'].to(device)
y_base_train = loaded_dataset['y_base_train'].to(device)
x_additional = loaded_dataset['x_additional'].to(device)
y_additional = loaded_dataset['y_additional'].to(device)
x_test = loaded_dataset['x_test'].to(device)
y_test = loaded_dataset['y_test'].to(device)

# Loss function and metrics already grabbed from the model module
analyze_wiggles_metrics(
    model_list = all_models, 
    x_base_train = x_base_train,
    y_base_train = y_base_train, 
    x_additional = x_additional,
    y_additional = y_additional,
    x_test = x_test,
    y_test = y_test, 
    dataset_quantities = dataset_quantities, 
    dataset_type = dataset_type, 
    metrics = {"loss": loss_fn, **other_metrics}, 
    coefficients = coefficients,
    num_directions = num_directions,
    perturbation_seed = perturbation_seed,
    base_output_dir = base_output_dir,
    device = device,  # can be set to GPU if needed
)


""" Our saved results are structured as follows:
wiggle_results: List of dictionaries containing wiggle test results
Each dictionary is of the form
{
'loss':
'coefficients':
'accs':
'perturbation_seed':
'perturbation_norm':
}
model: PyTorch model used in analysis (state_dict will be saved)
output_dir: Directory to save results (default: "imgs/swiss/random_dirs")
filename: Name of output file (default: "random_directions.npz")
**kwargs: Additional key-value pairs to be saved in the output file
Typically:
'additional_data':
'model_trained_data':
'dataset_type':
'base_dataset_size': 
'test_loss':
'test_accs':
'num_params':
"""

The number of parameters of the perturbation is 235146
Testing on data with 0 samples - 500 directions
Testing model trained on 0 additional data.
Loss: 1.2318
Accs: 0.6785


Wiggle completed in 45.14 seconds for data model trained with 0 samples
Saved to data_0

Testing model trained on 1940 additional data.
Loss: 0.4537
Accs: 0.9132


Wiggle completed in 46.15 seconds for data model trained with 1940 samples
Saved to data_0

Testing model trained on 19940 additional data.
Loss: 0.1205
Accs: 0.9689


Wiggle completed in 47.77 seconds for data model trained with 19940 samples
Saved to data_0

Testing model trained on 540 additional data.
Loss: 0.6655
Accs: 0.8755


Wiggle completed in 48.02 seconds for data model trained with 540 samples
Saved to data_0

Testing model trained on 5940 additional data.
Loss: 0.2533
Accs: 0.9463


...

Wiggle completed in 47.01 seconds for data model trained with 59940 samples
Saved to data_5940

Testing on data with 19940 samples - 500 directions
Testing model trained on 0 additional data.
Testing model trained on 1940 additional data.
Testing model trained on 19940 additional data.
Loss: 0.1205
Accs: 0.9689


Wiggle completed in 72.44 seconds for data model trained with 19940 samples
Saved to data_19940

Testing model trained on 540 additional data.
Testing model trained on 5940 additional data.
Testing model trained on 59940 additional data.
Loss: 0.0670
Accs: 0.9800


Wiggle completed in 73.14 seconds for data model trained with 59940 samples
Saved to data_19940

Testing on data with 59940 samples - 500 directions
Testing model trained on 0 additional data.
Testing model trained on 1940 additional data.
Testing model trained on 19940 additional data.
Testing model trained on 540 additional data.
Testing model trained on 5940 additional data.
Testing model trained on 59940 additional data.
Loss: 0.0670
Accs: 0.9800


Wiggle completed in 167.09 seconds for data model trained with 59940 samples
Saved to data_59940



' Our saved results are structured as follows:\nwiggle_results: List of dictionaries containing wiggle test results\nEach dictionary is of the form\n{\n\'loss\':\n\'coefficients\':\n\'accs\':\n\'perturbation_seed\':\n\'perturbation_norm\':\n}\nmodel: PyTorch model used in analysis (state_dict will be saved)\noutput_dir: Directory to save results (default: "imgs/swiss/random_dirs")\nfilename: Name of output file (default: "random_directions.npz")\n**kwargs: Additional key-value pairs to be saved in the output file\nTypically:\n\'additional_data\':\n\'model_trained_data\':\n\'dataset_type\':\n\'base_dataset_size\': \n\'test_loss\':\n\'test_accs\':\n\'num_params\':\n'